# 保存和加载checkpoint
保存和加载通用的检查点模型以进行推断或恢复训练，这有助于您从上一个地方继续进行。  
当保存一个常规检查点时，您必须保存模型的state_dict之外的更多信息。  
保存优化器的state_dict也很重要，因为它包含缓冲区和参数，随着模型的运行而更新。  
您可能希望保存的其他项目是您离开的epoch，最新记录的训练损失，外部torch.nn.Embedding层，以及更多，这些取决于您自己的算法。

要保存多个检查点，必须将它们组织在字典中，并使用torch.save()序列化字典。  
一个常见的PyTorch约定是使用.tar文件扩展名保存这些检查点。  
要加载这些文件，首先初始化模型和优化器，然后使用torch.load()在本地加载字典。从这里开始，您可以通过查询字典轻松地访问保存的项目。

## 步骤
1、导入所有必要的库来加载我们的数据  
2、定义和初始化神经网络
3、初始化优化器  
4、保存常规checkpoint  
5、加载常规checkpoint  

In [1]:
# 1、导入所有必要的库
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
# 2、定义和初始化神经网络
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()
print(net)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [3]:
# 3、初始化优化器
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [4]:
# 4、保存通用checkpoint
EPOCH = 5
PATH = "model.pt"
LOSS = 0.4

torch.save({
            'epoch': EPOCH,
            'model_state_dict': net.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': LOSS,
            }, PATH)

In [ ]:
# 5、加载通用checkpoint
model = Net()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model.eval()
# - or -
model.train()